In [2]:
from mlflow.tracking import MlflowClient

In [3]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [4]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [6]:
client.search_experiments()

[<Experiment: artifact_location='/Users/Jake/Olvin/dev/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1682585772380, experiment_id='1', last_update_time=1682585772380, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [7]:
client.create_experiment("my-test-experiment")

'2'

In [8]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids="1",
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [11]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse = {run.data.metrics['rmse']:.4f}")

run id: 8cd4bd90c153414996e8f26aea4a875d, rmse = 5.7852
run id: 6c29efb995de456691a402a9c2b3794b, rmse = 5.7852
run id: 98ec9262ecee4df39e7f14b8d2d874c1, rmse = 5.7852
run id: 9cdfff70fdf0430e8c546dcee236f1ea, rmse = 5.7946
run id: 32564d13d26446a7b0b01320c24ff194, rmse = 5.8020


In [28]:
model_name = "nyc-taxi-regressor"

In [12]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [14]:
run_id = runs[0].info.run_id
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name=model_name)

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/04/27 17:08:53 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1682611733848, current_stage='None', description=None, last_updated_timestamp=1682611733848, name='nyc-taxi-regressor', run_id='8cd4bd90c153414996e8f26aea4a875d', run_link=None, source='/Users/Jake/Olvin/dev/mlops-zoomcamp/02-experiment-tracking/mlruns/1/8cd4bd90c153414996e8f26aea4a875d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [19]:
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging
version: 3, stage: None


In [27]:
model_version = 3
model_stage = "Staging"

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=model_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1682611733848, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2023-04-27', last_updated_timestamp=1682612304666, name='nyc-taxi-regressor', run_id='8cd4bd90c153414996e8f26aea4a875d', run_link=None, source='/Users/Jake/Olvin/dev/mlops-zoomcamp/02-experiment-tracking/mlruns/1/8cd4bd90c153414996e8f26aea4a875d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [24]:
from datetime import datetime

In [26]:
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {model_stage} on {datetime.today().date()}"
)

<ModelVersion: aliases=[], creation_timestamp=1682611733848, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2023-04-27', last_updated_timestamp=1682612247764, name='nyc-taxi-regressor', run_id='8cd4bd90c153414996e8f26aea4a875d', run_link=None, source='/Users/Jake/Olvin/dev/mlops-zoomcamp/02-experiment-tracking/mlruns/1/8cd4bd90c153414996e8f26aea4a875d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [ ]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df['duration'] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime).dt.total_seconds() / 60
    df = df[((df.duration >= 1) & (df.duration <= 60))]
    categorical = ['PULocationID', 'DOLocationID']

    df[categorical] = df[categorical].astype(str)

    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + "_" + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient="records")
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=True)}